In [37]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False
if IN_COLAB:
    # Clone the entire repo to access the files.
    !git clone -l -s https://github.com/epfml/OptML_course.git cloned-repo
    %cd cloned-repo/labs/ex05/template/

Cloning into 'cloned-repo'...
remote: Enumerating objects: 2259, done.
remote: Counting objects: 100% (423/423), done.
remote: Compressing objects: 100% (219/219), done.
remote: Total 2259 (delta 208), reused 403 (delta 195), pack-reused 1836
Receiving objects: 100% (2259/2259), 425.21 MiB | 25.72 MiB/s, done.
Resolving deltas: 100% (1112/1112), done.
/content/cloned-repo/labs/ex05/template/cloned-repo/labs/ex05/template/cloned-repo/labs/ex05/template


In [36]:
import datetime
from helpers import *

height, weight, gender = load_data(sub_sample=False, add_outlier=False)
x, mean_x, std_x = standardize(height)
b, A = build_model_data(x, weight)

In [7]:
def full_objective(targets_b, data_A, params_x):
    """Compute the least squares objective over the whole dataset"""
    return 0.5 * np.mean(((data_A @ params_x) - targets_b)**2)

# Damped Newton

In [8]:
#Step of damped Newton
def damped_newton_update(targets_b, data_A, params_x, lambda_param):

  batch_size = len(targets_b)
  hessian = data_A.T@data_A
  grad = -data_A.T.dot(targets_b - data_A.dot(params_x)) / batch_size
  dx = np.linalg.solve(hessian + lambda_param*np.identity(hessian.shape[0]), grad)

  return dx

In [42]:
# Define the parameters of the algorithm.
max_iters = int(100)  # 10 passes through the dataset
batch_size = 20

# Initialization
x_initial = np.zeros(A.shape[1])

# Start SGD.
start_time = datetime.datetime.now()
damped_newton_objectives, damped_newton_xs = damped_newton(
    b, A, x_initial, batch_size, max_iters)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("Damped_newton: execution time={t:.3f} seconds".format(t=exection_time))

Damped_newton(0000/0099): objective =    2521.20
Damped_newton(0001/0099): objective =    2271.47
Damped_newton(0002/0099): objective =    2050.37
Damped_newton(0003/0099): objective =    1844.44
Damped_newton(0004/0099): objective =    1666.12
Damped_newton(0005/0099): objective =    1499.44
Damped_newton(0006/0099): objective =    1352.66
Damped_newton(0007/0099): objective =    1222.43
Damped_newton(0008/0099): objective =    1105.37
Damped_newton(0009/0099): objective =     995.87
Damped_newton(0010/0099): objective =     899.68
Damped_newton(0011/0099): objective =     810.49
Damped_newton(0012/0099): objective =     731.62
Damped_newton(0013/0099): objective =     666.44
Damped_newton(0014/0099): objective =     605.38
Damped_newton(0015/0099): objective =     546.46
Damped_newton(0016/0099): objective =     494.46
Damped_newton(0017/0099): objective =     445.92
Damped_newton(0018/0099): objective =     401.86
Damped_newton(0019/0099): objective =     363.87
Damped_newton(0020/0

In [43]:
def damped_newton(targets_b, data_A, initial_x, batch_size, max_iters):

  #Hyperparameter for damped newton
  lambda_param = 0.01

  xs = [initial_x]
  objectives = []
  x = initial_x

  for iteration in range(max_iters):

    indices = np.random.choice(len(targets_b), batch_size, replace=False)
    dx = damped_newton_update(targets_b[indices],data_A[indices],x,lambda_param)

    x = x - dx

    xs.append(x.copy())
    objective = full_objective(targets_b, data_A, x)
    objectives.append(objective)

    print("Damped_newton({bi:04d}/{ti:04d}): objective = {l:10.2f}".format(
                  bi=iteration, ti=max_iters - 1, l=objective))
  return objectives, xs




#Secant

In [40]:
def secant_update(targets_b, data_A, params_x, lambda_param, params_x_prev, hessian_inv_prev):

  batch_size = len(targets_b)

  grad = -data_A.T.dot(targets_b - data_A.dot(params_x)) / batch_size
  grad_prev = -data_A.T.dot(targets_b - data_A.dot(params_x_prev)) / batch_size

  sigma = params_x - params_x_prev
  y = grad - grad_prev
  H = hessian_inv_prev

  sigma = sigma.reshape((sigma.shape[0],1))
  y = y.reshape((y.shape[0],1))

  E = (1/y.T.dot(sigma))*((1 + (1/y.T.dot(sigma))*(y.T.dot(H.dot(y))))*(sigma.dot(sigma.T)) - H.dot(y.dot(sigma.T)) - sigma.dot(y.T.dot(H)))
  H_1 = H + E
  dx = H_1.dot(grad)

  return dx, H_1


In [44]:
def secant(targets_b, data_A, initial_x, batch_size, max_iters):

  xs = [initial_x]
  objectives = []
  x = initial_x

  H = np.linalg.inv(data_A.T@data_A)
  x = np.random.rand(x.shape[0])
  xs.append(x.copy())
  objective = full_objective(targets_b, data_A, x)
  objectives.append(objective)


  for iteration in range(max_iters):

    indices = np.random.choice(len(targets_b), batch_size, replace=False)
    dx, H = secant_update(targets_b[indices],data_A[indices],x,0,xs[-2],H)
    x = x - dx

    xs.append(x.copy())
    objective = full_objective(targets_b, data_A, x)
    objectives.append(objective)

    print("Secant({bi:04d}/{ti:04d}): objective = {l:10.2f}".format(
                  bi=iteration, ti=max_iters - 1, l=objective))
  return objectives, xs

In [45]:
# Define the parameters of the algorithm.
max_iters = int(100)  # 10 passes through the dataset
batch_size = 20

# Initialization
x_initial = np.zeros(A.shape[1])

# Start SGD.
start_time = datetime.datetime.now()
secant_objectives, secant_xs = secant(
    b, A, x_initial, batch_size, max_iters)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("Secant: execution time={t:.3f} seconds".format(t=exection_time))

Secant(0000/0099): objective =      88.97
Secant(0001/0099): objective =      88.52
Secant(0002/0099): objective =     103.84
Secant(0003/0099): objective =      96.39
Secant(0004/0099): objective =      98.93
Secant(0005/0099): objective =      88.68
Secant(0006/0099): objective =      88.31
Secant(0007/0099): objective =      90.25
Secant(0008/0099): objective =      88.25
Secant(0009/0099): objective =      90.17
Secant(0010/0099): objective =      89.60
Secant(0011/0099): objective =      88.10
Secant(0012/0099): objective =      88.11
Secant(0013/0099): objective =      88.09
Secant(0014/0099): objective =      88.19
Secant(0015/0099): objective =      89.52
Secant(0016/0099): objective =      88.54
Secant(0017/0099): objective =      88.95
Secant(0018/0099): objective =      88.12
Secant(0019/0099): objective =      89.07
Secant(0020/0099): objective =      92.05
Secant(0021/0099): objective =      88.22
Secant(0022/0099): objective =      89.26
Secant(0023/0099): objective =    